In [ ]:
#for referece

# Fs = 8000
# f = 5
# sample = 8000
# x = np.arange(sample)
# y = np.sin(2 * np.pi * f * x / Fs)
# plt.plot(x, y)
# plt.xlabel('sample(n)')
# plt.ylabel('voltage(V)')
# plt.show()

In [ ]:
# #create data points::

# seq_len = 31
# def create_time_series(seq_len):
#   freq = (np.random.random()*0.5) + 0.1  # 0.1 to 0.6
#   ampl = np.random.random() + 0.5  # 0.5 to 1.5
#   x = np.sin(np.arange(0,seq_len) * freq) * ampl
#   return x

## tflearn rnn experiment
use with tensorlab enviorment

tasks:

    [ ] create data set for training
        [ ] sin
        [ ] sales tpv day
    
    [ ] plot sin

    

In [2]:
import numpy as np
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
from tflearn.data_utils import shuffle

%matplotlib inline

In [4]:
#create trainX and train Y

np_train_exp_X = np.array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43], [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46], [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54], [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55], [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56], [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57], [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]], np.int32)
#print(np_train_exp_X)
np_train_exp_Y = np.arange(30, 60)
print(np_train_exp_Y)

[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54
 55 56 57 58 59]


In [105]:
#reshape trainX
np_train_exp_X = np.reshape(np_train_exp_X, (30, 30,1))/10

#reshape trainY
np_train_exp_Y = np.reshape(np_train_exp_Y, (30,1))/10
np_train_exp_Y.shape
#np_train_exp_X.shape

(30, 1)

In [106]:
#create TestX and TestY

np_test_exp_X = np.array([[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60], [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61], [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62], [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65], [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66], [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67], [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]], np.int32) 
np_test_exp_X = np.reshape(np_test_exp_X, (10, 30,1))/10

#print(np_test_exp_X)
np_test_exp_Y = np.arange(60,70)
np_test_exp_Y = np.reshape(np_test_exp_Y, (10,1))/10


In [107]:
np_test_exp_X

array([[[ 3. ],
        [ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 3.8],
        [ 3.9],
        [ 4. ],
        [ 4.1],
        [ 4.2],
        [ 4.3],
        [ 4.4],
        [ 4.5],
        [ 4.6],
        [ 4.7],
        [ 4.8],
        [ 4.9],
        [ 5. ],
        [ 5.1],
        [ 5.2],
        [ 5.3],
        [ 5.4],
        [ 5.5],
        [ 5.6],
        [ 5.7],
        [ 5.8],
        [ 5.9]],

       [[ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 3.8],
        [ 3.9],
        [ 4. ],
        [ 4.1],
        [ 4.2],
        [ 4.3],
        [ 4.4],
        [ 4.5],
        [ 4.6],
        [ 4.7],
        [ 4.8],
        [ 4.9],
        [ 5. ],
        [ 5.1],
        [ 5.2],
        [ 5.3],
        [ 5.4],
        [ 5.5],
        [ 5.6],
        [ 5.7],
        [ 5.8],
        [ 5.9],
        [ 6. ]],

       [[ 3.2],
        [ 3.3],
    

In [108]:
# x = np.array ([0,]*n)
# x = np.array ([])

In [109]:
# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    # Include the input layer, hidden layer(s), and set how you want to train the model
    
    # input layer
    
    net = tflearn.input_data(shape=[None, 30,1])
    #net = tflearn.embedding(net, input_dim=1, output_dim=60)
    # net = tflearn.lstm(net, n_units=30, return_seq=True)
    net = tflearn.lstm(net, n_units=64, return_seq=False)
    net = tflearn.fully_connected(net, 256, activation='linear')
    net = tflearn.fully_connected(net, 64, activation='linear')
    net = tflearn.fully_connected(net, 1, activation='linear')
    
    #Output Layer
    net = tflearn.regression(net, optimizer='adam', loss='mean_square')
    
    # This model assumes that your network is named "net"  
    #model = tflearn.DNN(net, clip_gradients=0.0, tensorboard_verbose=0)
    model = tflearn.DNN(net, clip_gradients=5.0)
    #model = tflearn.DNN(net)
    return model

In [110]:
model = build_model()

In [111]:
# Training
model.fit(np_train_exp_X, np_train_exp_Y, n_epoch=200, validation_set=0.1, show_metric=True, batch_size=10, shuffle=True)
#model.fit(train_features, train_next_point, n_epoch=1, validation_set=0.1, show_metric=True, batch_size=1)


Training Step: 599  | total loss: 0.00000 | time: 0.036s
| Adam | epoch: 200 | loss: 0.00000 - binary_acc: 0.0000 -- iter: 20/27
Training Step: 600  | total loss: 0.00000 | time: 1.060s
| Adam | epoch: 200 | loss: 0.00000 - binary_acc: 0.0000 | val_loss: 0.00000 - val_acc: 0.0000 -- iter: 27/27
--


In [112]:
# Compare the labels that our model predicts with the actual labels

# Find the indices of the most confident prediction for each item. That tells us the predicted digit for that sample.
predictions = np.array(model.predict(np_test_exp_X))

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = np_test_exp_Y
test_accuracy = np.mean(predictions == actual, axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  [ 0.]


In [113]:
predictions

array([[ 0.00080854],
       [ 0.00081237],
       [ 0.00081604],
       [ 0.00081958],
       [ 0.00082297],
       [ 0.00082623],
       [ 0.00082936],
       [ 0.00083236],
       [ 0.00083523],
       [ 0.00083799]], dtype=float32)

In [114]:
actual

array([[ 6. ],
       [ 6.1],
       [ 6.2],
       [ 6.3],
       [ 6.4],
       [ 6.5],
       [ 6.6],
       [ 6.7],
       [ 6.8],
       [ 6.9]])